# Step 1: Build ML Dataset

This notebook extracts FFT features from all images and creates a CSV dataset for machine learning.

## What We'll Do:
1. Load all damaged and undamaged images
2. Extract 12 FFT features from each image
3. Label them (damaged=1, undamaged=0)
4. Save to CSV file

## Why These Features?
- **Statistical features (mean, std, skewness, kurtosis)**: Capture overall distribution
- **Frequency bands (low, high, ratio)**: Identify noise vs content
- **Radial profile**: Track how frequencies change from center to edge

**Damaged images** typically have:
- Higher high-frequency energy (scratches, noise)
- Higher energy ratio (more noise relative to signal)
- Irregular radial profile

## Setup

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

# Import our feature extractor
from src.ml.feature_extractor import extract_ml_features

print("✓ Imports successful!")
print(f"Project root: {project_root}")

✓ Imports successful!
Project root: d:\R&D Project\image_processing


## Define Paths

In [2]:
# Dataset paths
data_path = os.path.join(project_root, "data/raw/AI_for_Art_Restoration_2/paired_dataset_art")
damaged_dir = os.path.join(data_path, "damaged")
undamaged_dir = os.path.join(data_path, "undamaged")

# Output path
output_dir = os.path.join(project_root, "data/processed")
os.makedirs(output_dir, exist_ok=True)
output_csv = os.path.join(output_dir, "fft_features.csv")

# Check if directories exist
print(f"Damaged directory exists: {os.path.exists(damaged_dir)}")
print(f"Undamaged directory exists: {os.path.exists(undamaged_dir)}")

# Count files
damaged_files = [f for f in os.listdir(damaged_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
undamaged_files = [f for f in os.listdir(undamaged_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

print(f"\nFound {len(damaged_files)} damaged images")
print(f"Found {len(undamaged_files)} undamaged images")
print(f"Total: {len(damaged_files) + len(undamaged_files)} images")

Damaged directory exists: True
Undamaged directory exists: True

Found 114 damaged images
Found 114 undamaged images
Total: 228 images


## Extract Features from All Images

This will take a few minutes depending on how many images you have.

**What's happening:**
- Loop through each image
- Apply FFT transform
- Extract 12 features
- Store in list with label

In [ ]:
# Lists to store features and labels
all_features = []
all_labels = []
all_filenames = []
feature_names = None

print("Processing damaged images...")
for filename in tqdm(damaged_files):
    try:
        path = os.path.join(damaged_dir, filename)
        features, names = extract_ml_features(path)
        
        all_features.append(features)
        all_labels.append(1)  # 1 = damaged
        all_filenames.append(filename)
        
        if feature_names is None:
            feature_names = names
            
    except Exception as e:
        print(f"Error processing {filename}: {e}")

print("\nProcessing undamaged images...")
for filename in tqdm(undamaged_files):
    try:
        path = os.path.join(undamaged_dir, filename)
        features, names = extract_ml_features(path)
        
        all_features.append(features)
        all_labels.append(0)  # 0 = undamaged
        all_filenames.append(filename)
        
    except Exception as e:
        print(f"Error processing {filename}: {e}")

print(f"\n✓ Successfully processed {len(all_features)} images")

## Create DataFrame and Save to CSV

In [ ]:
# Convert to DataFrame
df = pd.DataFrame(all_features, columns=feature_names)
df['label'] = all_labels
df['filename'] = all_filenames

# Reorder columns (put filename and label first)
cols = ['filename', 'label'] + feature_names
df = df[cols]

# Save to CSV
df.to_csv(output_csv, index=False)

print(f"✓ Saved dataset to: {output_csv}")
print(f"\nDataset shape: {df.shape}")
print(f"Features: {len(feature_names)}")
print(f"Samples: {len(df)}")

## Preview the Dataset

In [ ]:
# Show first few rows
print("First 5 rows:")
display(df.head())

# Show label distribution
print("\nLabel distribution:")
print(df['label'].value_counts())
print(f"\n0 = Undamaged: {(df['label']==0).sum()} images")
print(f"1 = Damaged: {(df['label']==1).sum()} images")

## Basic Statistics

Let's see if damaged and undamaged images have different feature values.

In [ ]:
# Compare damaged vs undamaged
damaged_df = df[df['label'] == 1]
undamaged_df = df[df['label'] == 0]

print("Feature Comparison (Damaged vs Undamaged)")
print("=" * 70)
print(f"{'Feature':<20} {'Damaged (mean)':>15} {'Undamaged (mean)':>18} {'Difference':>12}")
print("-" * 70)

for feature in feature_names:
    damaged_mean = damaged_df[feature].mean()
    undamaged_mean = undamaged_df[feature].mean()
    diff = damaged_mean - undamaged_mean
    
    print(f"{feature:<20} {damaged_mean:>15.4f} {undamaged_mean:>18.4f} {diff:>12.4f}")

print("\n💡 Key Observations:")
print("   Look for features with large differences - these are most useful for classification!")
print("   Typically:")
print("   - high_freq_energy: Higher for damaged (scratches/noise)")
print("   - energy_ratio: Higher for damaged (more noise relative to signal)")

## Visualize Feature Distributions

In [ ]:
import matplotlib.pyplot as plt

# Plot distributions of key features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distributions: Damaged vs Undamaged', fontsize=16)

key_features = ['high_freq_energy', 'energy_ratio', 'low_freq_energy', 'std_dev']

for idx, feature in enumerate(key_features):
    ax = axes[idx // 2, idx % 2]
    
    ax.hist(damaged_df[feature], bins=30, alpha=0.5, label='Damaged', color='red')
    ax.hist(undamaged_df[feature], bins=30, alpha=0.5, label='Undamaged', color='blue')
    
    ax.set_xlabel(feature)
    ax.set_ylabel('Count')
    ax.set_title(f'{feature} Distribution')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("   - If histograms are well-separated → feature is good for classification")
print("   - If histograms overlap heavily → feature is less useful")

## Summary

✅ **What We Accomplished:**
1. Extracted 12 FFT features from all images
2. Created labeled dataset (damaged=1, undamaged=0)
3. Saved to CSV for ML training
4. Visualized feature distributions

📊 **Dataset Ready!**
- File: `data/processed/fft_features.csv`
- Features: 12 per image
- Labels: Binary (damaged/undamaged)

🎯 **Next Step:**
Open notebook `2_train_classifier.ipynb` to train a machine learning model!